<a href="https://colab.research.google.com/github/eduardofae/RL/blob/main/AT-04/04%20-%20Monte%20Carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Métodos de Monte Carlo

Este Colab foi preparado para que você implemente um algoritmo de Monte Carlo em um ambiente Gridworld 4x3.

Você irá:

1. **Implementar o algoritmo de first-visit Monte Carlo control**: Preencher as partes faltando na classe `MonteCarloAgent` para inicializar o agente, atualizar estimativas a partir de uma trajetória num episódio e treinar o agente, gerando as trajetórias.
3. **Visualizar os resultados**: Utilizar as ferramentas de visualização fornecidas para inspecionar os valores dos estados e a política resultante.

Ao final, você terá uma implementação funcional de um agente de Monte Carlo e uma compreensão de como ele pode ser aplicado para encontrar a política ótima em um ambiente com dinâmica desconhecida.

## Grid 4x3

O código abaixo deve ser preenchido com a implementação do grid 4x3, com as mesmas especificações do Colab de [MDPs](https://colab.research.google.com/drive/1iNG9EX1-piXQvmN4-wQzncBv4EX7-yZV?usp=sharing). Portanto, você pode/deve aproveitar o código já feito. Não há mais aquelas funções adicionais para permitir que o ambiente seja "consultado" para execução de métodos de programação dinâmica, porque agora o agente não vai conhecer a dinâmica do ambiente. Ele irá interagir somente pela API padrão do gymnasium.


### Especificação do GridWorld (igual à tarefa de MDPs)

A célula abaixo tem a especificação do grid 4x3, igual no colab de MDPs. Pode pular se você já tiver feito.



Implemente o ambiente do grid 4x3, preenchendo as células abaixo. Você deve permitir ao usuário especificar a recompensa de cada passo (padrão = -0.04), a probabilidade de 'escorregar' (padrão = 0.2) e o numero máximo de passos antes de encerrar o episódio. O espaço de ações deve ser discreto, com 4 ações (0=UP, 1=RIGHT, 2=DOWN, 3=LEFT), e o de estados também será discreto, com 12 estados (mesmo o estado 'parede' pode ser considerado nesta contagem). A convenção para numeração dos estados é (G=goal, #=parede, P=pit/buraco,S=start):
```
y=2    +----+----+----+----+
       |  8 |  9 | 10 | 11G|
       +----+----+----+----+
y=1    |  4 |  5#|  6 |  7P|
       +----+----+----+----+
y=0    |  0S|  1 |  2 |  3 |
       +----+----+----+----+
        x=0   x=1   x=2   x=3
```

Note que há métodos para converter a numeração de estados para as coordenadas x,y

## Código do GridWorld 4x3

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from typing import Optional,Iterable,Tuple
import matplotlib.pyplot as plt


class GridWorld4x3(gym.Env):
    metadata = {"render_modes": ["ansi"]}

    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 4

    def __init__(
        self,
        reward_step: float = -0.04,
        slip: float = 0.2,
        max_steps: int = 1000,
        seed: Optional[int] = None,
        render_mode: Optional[str] = None,
    ):
        super().__init__()
        super().__init__()
        self.reward_step = reward_step
        self.slip = slip
        self.seed = seed

        # Grid Description
        self.grid_size = (4,3)
        self.wall  = [5]
        self.goal  = [11]
        self.pit   = [7]
        self.start = 0

        # Rewards
        self.goal_reward = 1
        self.pit_reward = -1

        # Agent Position
        self.agent_state = self.start

        # Definition of action space
        self.n_actions = 4
        self.action_space = spaces.Discrete(self.n_actions)

        # Definition of observation space
        self.n_states = self.grid_size[0]*self.grid_size[1]
        self.observation_space = spaces.Discrete(self.n_states)

        # COISAS PARA O VIZ
        self.wall_pos = (1,1)
        self.goal_pos = (3,2)
        self.pit_pos = (3,1)
        self.start_pos = (0,0)
        self.nrows = self.grid_size[1]
        self.ncols = self.grid_size[0]

        # Update Render Mode
        if render_mode:
            self.metadata = {"render_modes": [render_mode]}

    # ---------- conversão estado/posição ----------
    def pos_to_state(self, pos):
        x, y = pos
        return y * self.grid_size[0] + x

    def state_to_pos(self, s):
        return (s % self.grid_size[0], s // self.grid_size[0])

    # ---------- API Gym ----------
    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)

        # Reset agent position and step count
        self.agent_state = self.start

        # Returns
        obs = self.agent_state
        info = {}
        return obs, info

    def is_vertical(self, action: int):
        return action == self.UP or action == self.DOWN

    def handle_slip(self, action: int):
        direction = 1 if np.random.rand() < 0.5 else -1
        action = action + direction
        return action%4 if action != -1 else 3

    def get_next_state(self, state: int, action: int):
        x, y = self.state_to_pos(state)

        if self.is_vertical(action):
            new_y = np.clip(y+(1-action), 0, self.grid_size[1]-1)
            if self.pos_to_state((x,new_y)) not in self.wall:
                y = new_y
        else:
            new_x = np.clip(x+(2-action), 0, self.grid_size[0]-1)
            if self.pos_to_state((new_x,y)) not in self.wall:
                x = new_x

        return self.pos_to_state((x, y))

    def get_reward(self, state: int):
        if state in self.goal:
            return self.goal_reward
        if state in self.pit:
            return self.pit_reward
        return self.reward_step

    def step(self, action: int):
        # Handle slip
        if np.random.rand() < 0.2:
            action = self.handle_slip(action)

        # Update position
        self.agent_state = self.get_next_state(self.agent_state, action)

        # Checks if goal state reached
        terminated = self.agent_state in self.goal or self.agent_state in self.pit

        # Calculates the reward
        reward = self.get_reward(self.agent_state)

        # Not using
        info = {}

        # Returns the agent state
        obs = self.agent_state

        # Ends the episode when max steps reached
        truncated = False
        return obs, reward, terminated, truncated, info

    def render(self, mode=None):
        div = ' ' * 7 + '+----' * self.grid_size[0] + '+'
        for i in range(self.grid_size[1]):
            y = self.grid_size[1]-i-1
            print(div)
            print(f'y={y:<5}', end='')
            for x in range(self.grid_size[0]):
                state = self.pos_to_state((x,y))
                state_class = 'G' if state in self.goal else '#' if state in self.wall else 'P' if state in self.pit else 'S' if state == self.start else ''
                state_print = (str(state) if self.agent_state != state else 'X') + state_class
                print(f'|{state_print:^4}', end='')
            print('|')
        print(div)
        print('        ', end='')
        for x in range(self.grid_size[0]):
            print(f'x={x:<2} ', end='')

    def close(self):
      pass # nao precisa implementar


## Codigo pra testar o Env

A célula abaixo faz o teste básico pra verificar que o ambiente respeita a interface gymnasium

In [ ]:
# Deve continuar passando nos testes do Gymnasium
from gymnasium.utils.env_checker import check_env

# Criar uma instância do ambiente
env = GridWorld4x3()

# This will catch many common issues
try:
    check_env(env)
    print("Environment passes all checks!")
except Exception as e:
    print(f"Environment has issues: {e}")

Environment passes all checks!


/usr/local/lib/python3.12/dist-packages/gymnasium/utils/env_checker.py:434: UserWarning: WARN: Not able to test alternative render modes due to the environment not having a spec. Try instantiating the environment through `gymnasium.make`
  logger.warn(


## Tarefa: Agente de Monte Carlo

Agora você irá implementar o algoritmo de first-visit Monte Carlo control. Você irá trabalhar na classe `MonteCarloAgent` fornecida abaixo. Seu objetivo é preencher as partes faltando para que o agente seja capaz de escolher ações e atualizar estimativas de valor para encontrar a política ótima em um ambiente. No caso deste enunciado será o grid 4x3.

Você irá:

1.  **Implementar a inicialização (`__init__`)**: Prepare o agente para manter o registro dos valores-Q e os retornos para cada par estado-ação visitado durante sua "vida".
2.  **Implementar `greedy_action(state)`**: Preencha este método para retornar a ação gulosa para um dado estado, com base nos valores-Q correntes.
3.  **Implementar `updateQ(episode)`**: Complete este método para receber um episódio completo e atualizar os valores-Q usando a abordagem de First-Visit Monte Carlo. Esta função não deve resetar o agente. Você deve "continuar" a atualização a partir dos valores-Q atuais. Você pode testar esta função independente da `train`, já que esta recebe um episódio pronto.
4.  **Implementar `train(n_episodes)`**: Desenvolva este método para executar múltiplos episódios utilizando uma política epsilon-gulosa, coletando os dados de cada episódio e utilizando a função `updateQ` para treinar o agente. Esta função também não deve resetar o agente. Você deve "continuar" o treino de onde tiver parado.

Ao final, você terá um agente capaz de aprender a política ótima do ambiente Gridworld 4x3 utilizando o algoritmo de Monte Carlo first-visit.

In [ ]:
import numpy as np
import gymnasium as gym
from collections import defaultdict


class MonteCarloAgent:
    def __init__(self, env: gym.Env, gamma: float = 0.9, epsilon: float = 0.1) -> None:
        """
        Alem das variaveis triviais, inicialize os valores-Q como zero pra todos s,a
        e as listas de retorno para cada par estado-ação (ou contagens, se for fazer a
        atualização incremental).
        """
        self.env = env
        self.gamma = gamma
        self.epsilon = epsilon
        self.Q_values = np.zeros((self.env.n_states, self.env.n_actions), dtype=np.float64)
        self.sa_count = np.zeros((self.env.n_states, self.env.n_actions), dtype=np.int64)


    def updateQ(self, episode: list[tuple[int, int, float]]) -> None:
        """
        Atualiza Q(s,a) usando First-Visit Monte Carlo a partir de um episódio completo.
        O episodio é uma lista de tuplas (estado, ação, recompensa), indicando a
        trajetória do agente naquele episódio. O estado terminal não estará nesta lista,
        pois a última ação e recompensa válidas foi feita/recebida antes dele.

        Esta função NÃO deve resetar o agente. Se ela for chamada consecutivamente, as
        atualizações devem ser cumulativas.
        """
        G = 0
        episode.reverse()
        sa_episode = [(state, action) for state, action, _ in episode]
        for i, (state, action, reward) in enumerate(episode):
            G = self.gamma * G + reward
            if (state, action) in sa_episode[i+1:]: continue
            self.sa_count[state][action] += 1
            self.Q_values[state][action] += (G - self.Q_values[state][action]) / self.sa_count[state][action]

    def train(self, n_episodes: int) -> None:
        """
        Executa o treino do agente pelo numero de episodios especificado.
        Para cada episodio, reseta o ambiente, segue a politica epsilon-gulosa,
        gerando a trajetoria até o estado terminal e atualiza as estimativas de Q

        Esta função NÃO deve resetar o agente. Se ela for chamada consecutivamente,
        as sessões de treino devem ser cumulativas.
        """
        for i in range(n_episodes):
            state, _ = self.env.reset()
            episode = []
            while True:
                if np.random.rand() < self.epsilon:
                    action = np.random.randint(self.env.n_actions)
                else:
                    action = self.greedy_action(state)
                new_state, reward, terminated, truncated, _ = self.env.step(action)
                episode.append((state, action, reward))
                if terminated or truncated: break
                state = new_state
            self.updateQ(episode)

    def V(self, state: int) -> float:
        """Retorna V(s) = max_a Q(s,a)"""
        return max(self.Q_values[state])

    def Q(self, state: int, action: int) -> float:
        """Retorna o valor-Q de um par estado-ação"""
        return self.Q_values[state][action]

    def greedy_action(self, state: int) -> int:
        """Retorna a ação de maior valor para o estado recebido"""
        best_reward = float('-inf')
        for action in range(self.env.n_actions):
            reward = self.Q(state, action)
            if reward > best_reward:
                best_reward = reward
                best_action = action
        return best_action


## Utilitário para visualizar o agente

O mesmo da tarefa anterior, porém "enxergando" um pouco menos dos detalhes internos do env

### Código do utilitário

In [ ]:
class AgentVisualizer:
    def __init__(self, agent, env):
        """
        agent: objeto que implementa V(s), Q(s,a) e greedy_action(s))
        env: GridWorld4x3-like (tem nrows, ncols, pos_to_state, state_to_pos, is_terminal, start_pos, goal_pos, pit_pos, wall_pos)
        """
        self.agent = agent
        self.env = env
        self.action_to_str = {0: "↑", 1: "→", 2: "↓", 3: "←"}

        # Precompute special states
        self.wall_s = self.env.pos_to_state(self.env.wall_pos)
        self.start_s = self.env.pos_to_state(self.env.start_pos)
        self.goal_s = self.env.pos_to_state(self.env.goal_pos)
        self.pit_s = self.env.pos_to_state(self.env.pit_pos)

    # -----------------------
    # Política (setas)
    # -----------------------
    def print_policy(self):
        rows, cols = 3,4
        horiz = "+" + "+".join(["------"] * cols) + "+"

        for y in reversed(range(rows)):
            print(horiz)
            cells = []
            for x in range(cols):
                s = self.env.pos_to_state((x, y))
                if s == self.wall_s:
                    content = "##"
                elif s == self.goal_s:
                    content = " G "
                elif s == self.pit_s:
                    content = " P "
                else:
                    a = self.agent.greedy_action(s)
                    arrow = self.action_to_str.get(a, "?")
                    if s == self.start_s:
                        content = f"S{arrow}"
                    else:
                        content = arrow
                cells.append(f"{content:^6}")
            print("|" + "|".join(cells) + "|")
        print(horiz)

    # -----------------------
    # Valores V(s)
    # -----------------------
    def print_values(self):
        rows, cols = 3,4
        horiz = "+" + "+".join(["--------"] * cols) + "+"

        for y in reversed(range(rows)):
            print(horiz)
            cells = []
            for x in range(cols):
                s = self.env.pos_to_state((x, y))
                if s == self.wall_s:
                    content = "####"
                else:
                    v = self.agent.V(s)
                    if s == self.goal_s:
                        content = f"G({v:.2f})"
                    elif s == self.pit_s:
                        content = f"P({v:.2f})"
                    else:
                        content = f"{v:6.2f}"
                cells.append(f"{content:^8}")
            print("|" + "|".join(cells) + "|")
        print(horiz)

    # -----------------------
    # Q-values
    # -----------------------
    def print_qvalues(self):
        rows, cols = 3,4
        horiz = "+" + "+".join(["---------------"] * cols) + "+"

        for y in reversed(range(rows)):
            print(horiz)
            # três linhas por célula
            line1, line2, line3 = [], [], []
            for x in range(cols):
                s = self.env.pos_to_state((x, y))
                if s == self.wall_s:
                    c1 = "###############"
                    c2 = "###############"
                    c3 = "###############"
                else:
                    qvals = [self.agent.Q(s, a) for a in range(4)]
                    best = int(np.argmax(qvals))
                    up = f"↑:{qvals[0]:.2f}"
                    left = f"←:{qvals[3]:.2f}"
                    right = f"→:{qvals[1]:.2f}"
                    down = f"↓:{qvals[2]:.2f}"
                    c1 = f"{up:^15}"
                    c2 = f"{left:<7}{right:>8}"
                    c3 = f"{down:^15}"
                line1.append(c1)
                line2.append(c2)
                line3.append(c3)

            # agora cada linha recebe delimitadores
            print("|" + "|".join(line1) + "|")
            print("|" + "|".join(line2) + "|")
            print("|" + "|".join(line3) + "|")
        print(horiz)


### Exemplo de uso - 1 episódio simples

Ao executar o código a seguir, a saída esperada por ter executado as ações (acima, abaixo, acima, acima, direita, direita, direita) com reward_step=-0.1, slip=0 e gamma = 1 é como abaixo.



```
Teste. Parametros: reward_step=-0.1, slip=0, gamma=1
=== Política ===
+------+------+------+------+
|  →   |  →   |  →   |  G   |
+------+------+------+------+
|  ↑   |  ##  |  ↑   |  P   |
+------+------+------+------+
|  S↑  |  ↑   |  ↑   |  ↑   |
+------+------+------+------+

=== V(s) ===
+--------+--------+--------+--------+
|   0.80 |   0.90 |   1.00 |G(0.00) |
+--------+--------+--------+--------+
|   0.70 |  ####  |   0.00 |P(0.00) |
+--------+--------+--------+--------+
|   0.40 |   0.00 |   0.00 |   0.00 |
+--------+--------+--------+--------+

=== Q(s,a) ===
+---------------+---------------+---------------+---------------+
|    ↑:0.00     |    ↑:0.00     |    ↑:0.00     |    ↑:0.00     |
|←:0.00   →:0.80|←:0.00   →:0.90|←:0.00   →:1.00|←:0.00   →:0.00|
|    ↓:0.00     |    ↓:0.00     |    ↓:0.00     |    ↓:0.00     |
+---------------+---------------+---------------+---------------+
|    ↑:0.70     |###############|    ↑:0.00     |    ↑:0.00     |
|←:0.00   →:0.00|###############|←:0.00   →:0.00|←:0.00   →:0.00|
|    ↓:0.50     |###############|    ↓:0.00     |    ↓:0.00     |
+---------------+---------------+---------------+---------------+
|    ↑:0.40     |    ↑:0.00     |    ↑:0.00     |    ↑:0.00     |
|←:0.00   →:0.00|←:0.00   →:0.00|←:0.00   →:0.00|←:0.00   →:0.00|
|    ↓:0.00     |    ↓:0.00     |    ↓:0.00     |    ↓:0.00     |
+---------------+---------------+---------------+---------------+
```

In [ ]:
env = GridWorld4x3(reward_step=-0.1, slip=0) # grid deterministico
agent = MonteCarloAgent(env, gamma=1) # agente sem desconto
viz = AgentVisualizer(agent, env)

#0=UP, 1=RIGHT, 2=DOWN, 3=LEFT

# politica simples: vai pra cima (0), baixo (2), sobe tudo(0), depois tudo pra direita (1)
simple_episode = [
    (0, 0, -0.1), #start, up
    (4, 2, -0.1), #2,1 - down
    (0, 0, -0.1), #start, up
    (4, 0, -0.1), #2,1 - up
    (8, 1, -0.1), #3,1 - right
    (9, 1, -0.1), #3,2 - right
    (10, 1, +1), #3,3 - right (done)
]

print(f"\nTeste. Parametros: reward_step={env.reward_step}, slip={env.slip}, gamma={agent.gamma}")
agent.updateQ(simple_episode)

print("=== Política ===")
viz.print_policy()

print("\n=== V(s) ===")
viz.print_values()

print("\n=== Q(s,a) ===")
viz.print_qvalues()

env.close()


Teste. Parametros: reward_step=-0.1, slip=0, gamma=1
=== Política ===
+------+------+------+------+
|  →   |  →   |  →   |  G   |
+------+------+------+------+
|  ↑   |  ##  |  ↑   |  P   |
+------+------+------+------+
|  S↑  |  ↑   |  ↑   |  ↑   |
+------+------+------+------+

=== V(s) ===
+--------+--------+--------+--------+
|   0.80 |   0.90 |   1.00 |G(0.00) |
+--------+--------+--------+--------+
|   0.70 |  ####  |   0.00 |P(0.00) |
+--------+--------+--------+--------+
|   0.40 |   0.00 |   0.00 |   0.00 |
+--------+--------+--------+--------+

=== Q(s,a) ===
+---------------+---------------+---------------+---------------+
|    ↑:0.00     |    ↑:0.00     |    ↑:0.00     |    ↑:0.00     |
|←:0.00   →:0.80|←:0.00   →:0.90|←:0.00   →:1.00|←:0.00   →:0.00|
|    ↓:0.00     |    ↓:0.00     |    ↓:0.00     |    ↓:0.00     |
+---------------+---------------+---------------+---------------+
|    ↑:0.70     |###############|    ↑:0.00     |    ↑:0.00     |
|←:0.00   →:0.00|##########

### Exemplo de uso - outro episódio simples

Ao executar o código a seguir, a saída esperada por ter executado as ações (acima, abaixo, acima, acima, direita, direita, direita) com reward_step=-0.1, slip=0 e gamma = 0.9 é como abaixo.



```
Teste. Parametros: reward_step=-0.1, slip=0, gamma=0.9
=== Política ===
+------+------+------+------+
|  →   |  →   |  →   |  G   |
+------+------+------+------+
|  ↑   |  ##  |  ↑   |  P   |
+------+------+------+------+
|  S↑  |  ↑   |  ↑   |  ↑   |
+------+------+------+------+

=== V(s) ===
+--------+--------+--------+--------+
|   0.62 |   0.80 |   1.00 |G(0.00) |
+--------+--------+--------+--------+
|   0.46 |  ####  |   0.00 |P(0.00) |
+--------+--------+--------+--------+
|   0.06 |   0.00 |   0.00 |   0.00 |
+--------+--------+--------+--------+

=== Q(s,a) ===
+---------------+---------------+---------------+---------------+
|    ↑:0.00     |    ↑:0.00     |    ↑:0.00     |    ↑:0.00     |
|←:0.00   →:0.62|←:0.00   →:0.80|←:0.00   →:1.00|←:0.00   →:0.00|
|    ↓:0.00     |    ↓:0.00     |    ↓:0.00     |    ↓:0.00     |
+---------------+---------------+---------------+---------------+
|    ↑:0.46     |###############|    ↑:0.00     |    ↑:0.00     |
|←:0.00   →:0.00|###############|←:0.00   →:0.00|←:0.00   →:0.00|
|    ↓:0.18     |###############|    ↓:0.00     |    ↓:0.00     |
+---------------+---------------+---------------+---------------+
|    ↑:0.06     |    ↑:0.00     |    ↑:0.00     |    ↑:0.00     |
|←:0.00   →:0.00|←:0.00   →:0.00|←:0.00   →:0.00|←:0.00   →:0.00|
|    ↓:0.00     |    ↓:0.00     |    ↓:0.00     |    ↓:0.00     |
+---------------+---------------+---------------+---------------+
```

In [ ]:
env = GridWorld4x3(reward_step=-0.1, slip=0) # grid deterministico
agent = MonteCarloAgent(env, gamma=0.9) # agente sem desconto
viz = AgentVisualizer(agent, env)

#0=UP, 1=RIGHT, 2=DOWN, 3=LEFT

# politica simples: vai pra cima (0), baixo (2), sobe tudo(0), depois tudo pra direita (1)
simple_episode = [
    (0, 0, -0.1), #start, up
    (4, 2, -0.1), #2,1 - down
    (0, 0, -0.1), #start, up
    (4, 0, -0.1), #2,1 - up
    (8, 1, -0.1), #3,1 - right
    (9, 1, -0.1), #3,2 - right
    (10, 1, +1), #3,3 - right (done)
]

print(f"\nTeste. Parametros: reward_step={env.reward_step}, slip={env.slip}, gamma={agent.gamma}")
agent.updateQ(simple_episode)

print("=== Política ===")
viz.print_policy()

print("\n=== V(s) ===")
viz.print_values()

print("\n=== Q(s,a) ===")
viz.print_qvalues()

env.close()


Teste. Parametros: reward_step=-0.1, slip=0, gamma=0.9
=== Política ===
+------+------+------+------+
|  →   |  →   |  →   |  G   |
+------+------+------+------+
|  ↑   |  ##  |  ↑   |  P   |
+------+------+------+------+
|  S↑  |  ↑   |  ↑   |  ↑   |
+------+------+------+------+

=== V(s) ===
+--------+--------+--------+--------+
|   0.62 |   0.80 |   1.00 |G(0.00) |
+--------+--------+--------+--------+
|   0.46 |  ####  |   0.00 |P(0.00) |
+--------+--------+--------+--------+
|   0.06 |   0.00 |   0.00 |   0.00 |
+--------+--------+--------+--------+

=== Q(s,a) ===
+---------------+---------------+---------------+---------------+
|    ↑:0.00     |    ↑:0.00     |    ↑:0.00     |    ↑:0.00     |
|←:0.00   →:0.62|←:0.00   →:0.80|←:0.00   →:1.00|←:0.00   →:0.00|
|    ↓:0.00     |    ↓:0.00     |    ↓:0.00     |    ↓:0.00     |
+---------------+---------------+---------------+---------------+
|    ↑:0.46     |###############|    ↑:0.00     |    ↑:0.00     |
|←:0.00   →:0.00|########

### Treino (1000 episódios) e visualização do agente

Com 1000 episódios de treino, o agente deve poder encontrar a trajetória correta a partir do estado inicial. Porém, não se assuste se a ação para alguns estados menos visitados não for encontrada.

In [ ]:
env = GridWorld4x3() # grid com parâmetros padrão
agent = MonteCarloAgent(env, gamma=0.9, epsilon=0.2) # agente com desconto e epsilon-greedy
viz = AgentVisualizer(agent, env)

print("Treinando agente por 1000 episódios...")
agent.train(1000)
print("Treino concluído.")

print("\n=== Política final ===")
viz.print_policy()

print("\n=== V(s) final ===")
viz.print_values()

print("\n=== Q(s,a) final ===")
viz.print_qvalues()

env.close()

Treinando agente por 1000 episódios...
Treino concluído.

=== Política final ===
+------+------+------+------+
|  →   |  →   |  →   |  G   |
+------+------+------+------+
|  ↑   |  ##  |  ↑   |  P   |
+------+------+------+------+
|  S↑  |  ←   |  ↑   |  ←   |
+------+------+------+------+

=== V(s) final ===
+--------+--------+--------+--------+
|   0.52 |   0.70 |   0.91 |G(0.00) |
+--------+--------+--------+--------+
|   0.39 |  ####  |   0.58 |P(0.00) |
+--------+--------+--------+--------+
|   0.26 |   0.16 |   0.30 |   0.52 |
+--------+--------+--------+--------+

=== Q(s,a) final ===
+---------------+---------------+---------------+---------------+
|    ↑:0.35     |    ↑:0.54     |    ↑:0.71     |    ↑:0.00     |
|←:0.35   →:0.52|←:0.37   →:0.70|←:0.48   →:0.91|←:0.00   →:0.00|
|    ↓:0.27     |    ↓:0.56     |    ↓:0.45     |    ↓:0.00     |
+---------------+---------------+---------------+---------------+
|    ↑:0.39     |###############|    ↑:0.58     |    ↑:0.00     |
|←:0.

## Bônus: explore sua implementação

Teste diversos parâmetros para verificar seu efeito nos valores e na política. Você pode comparar seus resultados daqui com os da sua implementação de iteração de valor, para ver como o agente de Monte Carlo tem mais "dificuldade" de obter a política ótima